This notebook is part of the `kikuchipy` documentation https://kikuchipy.org.
Links to the documentation won't work from the notebook.

# Geometrical EBSD simulations

In this tutorial, we will inspect and visualize the results from EBSD indexing
by plotting Kikuchi lines and zone axes onto an EBSD signal. We consider this a
*geometrical* EBSD simulation, since it is only positions of Kikuchi lines and
zone axes that are computed. These simulations are based on the work by Aimo
Winkelmann in the supplementary material to
<cite data-cite="britton2016tutorial">Britton et al. (2016)</cite>.

These simulations can be helpful when checking whether indexing results are
correct and for interpreting them. 

Let's import the necessary libraries

In [ ]:
# Exchange inline for notebook or qt5 (from pyqt) for interactive plotting
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np

from diffpy.structure import Atom, Lattice, Structure
from diffsims.crystallography import ReciprocalLatticeVector
import hyperspy.api as hs
import kikuchipy as kp
from orix.crystal_map import Phase
from orix.quaternion import Rotation


# Plotting parameters
plt.rcParams.update(
    {"figure.figsize": (10, 10), "font.size": 20, "lines.markersize": 10}
)

We'll inspect the indexing results of a small nickel EBSD dataset of 3 x 3
patterns

In [ ]:
s = kp.data.nickel_ebsd_small()  # Use kp.load("data.h5") to load your own data
s

Let's enhance the Kikuchi bands by removing the static and dynamic backgrounds

In [ ]:
s.remove_static_background()
s.remove_dynamic_background()

In [ ]:
_ = hs.plot.plot_images(
    s, axes_decor=None, label=None, colorbar=False, tight_layout=True
)

To project Kikuchi lines and zone axes onto our detector, we need

1. a description of the crystal phase

2. the set of Kikuchi bands to consider, e.g. the sets of planes {111}, {200},
   {220}, and {311}

3. the crystal orientations with respect to the reference frame

4. the position of the detector with respect to the sample, in the form of a
   sample-detector model which includes the sample and detector tilt and the
   projection center (shortes distance from the source point on the
   sample to the detector), given here as (PC$_x$, PC$_y$, PC$_z$)

## Prepare phase and reflector list

We'll store the crystal phase information in an
[orix.crystal_map.Phase](https://orix.readthedocs.io/en/stable/reference/generated/orix.crystal_map.Phase.html)
instance

In [ ]:
phase = Phase(
    space_group=225,
    structure=Structure(
        atoms=[Atom("Ni", [0, 0, 0])],
        lattice=Lattice(3.52, 3.52, 3.52, 90, 90, 90),
    ),
)

print(phase)
print(phase.structure)

We'll build up the reflector list using
[diffsims.crystallography.ReciprocalLatticeVector](https://diffsims.readthedocs.io/en/latest/reference.html#diffsims.crystallography.ReciprocalLatticeVector)

In [ ]:
ref = ReciprocalLatticeVector(
    phase=phase, hkl=[[1, 1, 1], [2, 0, 0], [2, 2, 0], [3, 1, 1]]
)
ref

We'll obtain the symmetrically equivalent vectors and plot each family of
vectors in a distinct colour in the stereographic projection

In [ ]:
ref = ref.symmetrise().unique()
ref.size

In [ ]:
ref.print_table()

In [ ]:
# Dictionary with {hkl} as key and indices into `ref` as values
hkl_sets = ref.get_hkl_sets()
hkl_sets

In [ ]:
hkl_colors = np.zeros((ref.size, 3))
for idx, color in zip(
    hkl_sets.values(),
    [
        [1, 0, 0],
        [0, 1, 0],
        [0, 0, 1],
        [0.75, 0, 0.75],
    ],  # Red, green, blue, magenta
):
    hkl_colors[idx] = color

In [ ]:
hkl_labels = []
for hkl in ref.hkl.round(0).astype(int):
    hkl_labels.append(str(hkl).replace("[", "(").replace("]", ")"))

In [ ]:
ref.scatter(c=hkl_colors, grid=True, ec="k", vector_labels=hkl_labels)

We can also plot the plane traces, i.e. the Kikuchi lines, in both hemispheres
(they are identical for Ni)

In [ ]:
ref.draw_circle(
    color=hkl_colors, hemisphere="both", figure_kwargs=dict(figsize=(15, 10))
)

## Specify rotations and detector-sample geometry

We will index the nine patterns using PyEBSDIndex, by first estimating the
projection center (PC) followed by Hough indexing. The initial estimate of the
PC is based on previous experiments on the same microscope.

<div class="alert alert-info">

Note

kikuchipy cannot depend on PyEBSDIndex at the moment, as PyEBSDIndex does not
support all the combinations of Python versions and operating systems that
kikuchipy does. To install PyEBSDIndex, see their
[installation instructions](https://pyebsdindex.readthedocs.io/en/latest/installation.html).

PyEBSDIndex supports indexing face centered and body centered cubic (FCC and BCC)
materials.

</div>

Alternatively, if PyEBSDIndex
is not available, we can use orientations determined from a previous dictionary
indexing (see the [pattern matching](pattern_matching.ipynb) tutorial): the nine
patterns are from two grains with orientations given in Euler angles of about
$(\phi_1, \Phi, \phi_2) = (258^{\circ}, 58^{\circ}, 1^{\circ})$ and
$(\phi_1, \Phi, \phi_2) = (292^{\circ}, 62^{\circ}, 182^{\circ})$.

In [ ]:
from pyebsdindex import ebsd_index, pcopt

In [ ]:
sig_shape = s.axes_manager.signal_shape[::-1]
indexer = ebsd_index.EBSDIndexer(
    vendor="KIKUCHIPY", sampleTilt=70, camElev=0, patDim=sig_shape
)

In [ ]:
pc0 = (0.4, 0.2, 0.5)
pc = pcopt.optimize(s.data.reshape((-1,) + sig_shape), indexer, pc0)
print(pc)

In [ ]:
data, *_ = indexer.index_pats(s.data.reshape((-1,) + sig_shape), PC=pc)
rot = Rotation(data[-1]["quat"]).reshape(3, 3)
rot

In [ ]:
# Run if PyEBSDIndex is unavailable
# pc = (0.42, 0.22, 0.50)
# sig_shape = s.axes_manager.signal_shape[::-1]

# grain1 = np.deg2rad((258, 58, 1))
# grain2 = np.deg2rad((292, 62, 182))
# rot = Rotation.from_euler(
#    [[grain1, grain2, grain2], [grain1, grain2, grain2], [grain1, grain2, grain2]]
# )
# rot

We describe the sample-detector model in an
[kikuchipy.detectors.EBSDDetector](../reference/generated/kikuchipy.detectors.EBSDDetector.rst)
instance. From Hough indexing we know the projection center to be, in the EDAX
TSL convention (see the [reference frame](reference_frames.rst) tutorial for the
various conventions and more details on the use of the sample-detector model),
$(x^{*}, y^{*}, z^{*}) = (0.421, 0.7794, 0.5049)$. The sample was tilted
$70^{\circ}$ about the microscope X direction towards the detector, and the
detector normal was orthogonal to the optical axis (beam direction)

In [ ]:
detector = kp.detectors.EBSDDetector(sig_shape, sample_tilt=70, pc=pc)
detector

Note that the projection center gets converted internally to the Bruker
convention.

## Create simulations on detector

Now we're ready to create geometrical simulations. We create simulations using
the
[kikuchipy.simulations.KikuchiPatternSimulator](../reference/generated/kikuchipy.simulations.KikuchiPatternSimulator.rst),
which takes the reflectors as input

In [ ]:
simulator = kp.simulations.KikuchiPatternSimulator(ref)

In [ ]:
sim = simulator.on_detector(detector, rot)

By passing the detector and crystal orientations to
[KikuchiPatternSimulator.on_detector()](../reference/generated/kikuchipy.simulations.KikuchiPatternSimulator.on_detector.rst),
we've obtained a
[kikuchipy.simulations.GeometricalKikuchiPatternSimulation](../reference/generated/kikuchipy.simulations.GeometricalKikuchiPatternSimulation.rst),
which stores the detector and gnomonic coordinates of the Kikuchi lines and
zone axes for each crystal orientation

In [ ]:
sim

We see that not all 50 of the reflectors in the reflector list are present in
some pattern.

## Plot simulations

These geometrical simulations can be plotted one-by-one by themselves

In [ ]:
sim.plot()

Or, they can be plotted on top of patterns in three ways: passing a pattern to
[GeometricalKikuchiPatternSimulation.plot()](../reference/generated/kikuchipy.simulations.GeometricalKikuchiPatternSimulation.plot.rst)

In [ ]:
sim.plot(index=(1, 2), pattern=s.inav[2, 1].data)

Or, we can obtain collections of lines, zone axes and zone axes labels as
Matplotlib objects via
[GeometricalKikuchiPatternSimulation.as_collections()](../reference/generated/kikuchipy.simulations.GeometricalKikuchiPatternSimulation.as_collections.rst)
and add them to an existing Matplotlib axis

In [ ]:
fig, ax = plt.subplots(ncols=3, nrows=3, figsize=(15, 15))

for idx in np.ndindex(s.axes_manager.navigation_shape[::-1]):
    ax[idx].imshow(s.data[idx], cmap="gray")
    ax[idx].axis("off")

    lines, zone_axes, zone_axes_labels = sim.as_collections(
        idx,
        zone_axes=True,
        zone_axes_labels=True,
        zone_axes_labels_kwargs=dict(fontsize=12),
    )
    ax[idx].add_collection(lines)
    ax[idx].add_collection(zone_axes)
    for label in zone_axes_labels:
        ax[idx].add_artist(label)

fig.tight_layout()

Or, we can obtain the lines, zone axes, zone axes labels and PCs as HyperSpy
markers via
[GeometricalKikuchiPatternSimulation.as_markers()](../reference/generated/kikuchipy.simulations.GeometricalKikuchiPatternSimulation.as_markers.rst)
and add them to a signal of the same navigation shape as the simulation
instance. This enables navigating the patterns *with* the geometrical simulations

In [ ]:
markers = sim.as_markers()

# To delete previously added permanent markers, do
# del s.metadata.Markers

s.add_marker(markers, plot_marker=False, permanent=True)

In [ ]:
s.plot()